In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
X_OOT_scaled = pd.DataFrame(scaler.transform(X_OOT), columns=X_OOT.columns)

drop_index = ((X_train_scaled >= 6) | (X_train_scaled <= -6)).sum(axis=1)
drop_index = drop_index[drop_index >= 1].index.to_list()
X_train_scaled = X_train_scaled.drop(index=drop_index)
y_train = pd.DataFrame(y_train).iloc[X_train_scaled.index, :]['Fraud']

model = MLPClassifier(hidden_layer_sizes=(20),
                      max_iter=400,
                      solver='adam',
                      activation='relu')

y_neg_cnt = (y_train == 0).sum()
smote = SMOTE(sampling_strategy={0: y_neg_cnt, 1: int(y_neg_cnt/10)})
X_train_smo, y_train_smo = smote.fit_sample(X_train_scaled, y_train)

model.fit(X_train_smo, y_train_smo)

In [ ]:
y_train_pred_prob = model.predict_proba(X_train_scaled)[:,1]
y_test_pred_prob = model.predict_proba(X_test_scaled)[:,1]
y_OOT_pred_prob = model.predict_proba(X_OOT_scaled)[:,1]

In [ ]:
get_results(y_train, y_train_pred_prob)

In [ ]:
get_results(y_test, y_test_pred_prob)

In [ ]:
get_results(y_OOT, y_OOT_pred_prob)